<a href="https://colab.research.google.com/github/Abhi-ram-17/VRV-Security-Assessment/blob/main/Log_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
from collections import Counter

# Define constants and file paths
LOG_FILE = "/content/sample.log"  # Replace with your log file path
OUTPUT_CSV = "log_analysis_results.csv"
FAILED_LOGIN_THRESHOLD = 10

def parse_log_file(log_file):
    """Parse the log file to extract relevant information."""
    ip_requests = Counter()
    endpoints = Counter()
    failed_logins = Counter()

    with open(log_file, "r") as file:
        for line in file:
            parts = line.split()
            if len(parts) < 9:
                continue

            # Extract IP address
            ip_address = parts[0]
            ip_requests[ip_address] += 1

            # Extract endpoint
            request = parts[5][1:]  # Remove starting "
            if request in ("GET", "POST"):
                endpoint = parts[6]
                endpoints[endpoint] += 1

            # Check for failed login attempts
            status_code = parts[8]
            if status_code == "401" or "Invalid credentials" in line:
                failed_logins[ip_address] += 1

    return ip_requests, endpoints, failed_logins

def analyze_logs(ip_requests, endpoints, failed_logins):
    """Analyze parsed logs to generate results."""
    # Sort IP requests
    sorted_ip_requests = ip_requests.most_common()

    # Identify most frequently accessed endpoint
    most_accessed_endpoint = endpoints.most_common(1)

    # Detect suspicious activity
    suspicious_ips = {ip: count for ip, count in failed_logins.items() if count > FAILED_LOGIN_THRESHOLD}

    return sorted_ip_requests, most_accessed_endpoint, suspicious_ips

def save_to_csv_fixed(sorted_ip_requests, most_accessed_endpoint, suspicious_ips, output_csv):
    """Save analysis results to a CSV file with consistent formatting."""
    with open(output_csv, mode="w", newline="") as file:
        writer = csv.writer(file)

        # Write Requests per IP
        writer.writerow(["Section", "Detail 1", "Detail 2"])
        writer.writerow(["Requests per IP", "IP Address", "Request Count"])
        for ip, count in sorted_ip_requests:
            writer.writerow(["", ip, count])
        writer.writerow([])

        # Write Most Accessed Endpoint
        writer.writerow(["Most Accessed Endpoint", "Endpoint", "Access Count"])
        if most_accessed_endpoint:
            endpoint, count = most_accessed_endpoint[0]
            writer.writerow(["", endpoint, count])
        writer.writerow([])

        # Write Suspicious Activity
        writer.writerow(["Suspicious Activity", "IP Address", "Failed Login Count"])
        for ip, count in suspicious_ips.items():
            writer.writerow(["", ip, count])

def main():
    """Main function to execute log analysis."""
    ip_requests, endpoints, failed_logins = parse_log_file(LOG_FILE)
    sorted_ip_requests, most_accessed_endpoint, suspicious_ips = analyze_logs(ip_requests, endpoints, failed_logins)
    save_to_csv_fixed(sorted_ip_requests, most_accessed_endpoint, suspicious_ips, OUTPUT_CSV)
    print(f"Results saved to {OUTPUT_CSV}")

if __name__ == "__main__":
    main()


Results saved to log_analysis_results.csv
